In [ ]:
using Unitful
include("../src/AutoSolver.jl")

In [ ]:
# include("../../AstroKit.jl/src/AstroKit.jl")
include("../../SciBase.jl/src/SciBase.jl")

In [ ]:
# # Displacement
# disp(x0,x) = x-x0
# dispConstAcc1(v0,a,t) = v0*t+1/2*a*t^2
# dispConstAcc2(v0,v,t) = ((v+v0)/2)*t
# # Velocity
# avgVel(x0,x,t0,t) = (x-x0)/(t-t0)
# velConstAcc1(v0,a,t) = v0+a*t
# velConstAcc2(x0,x,v0,a) = sqrt(v0^2+2*a*(x-x0))
# # Acceleration
# accConst(x0,x,v0,v) = (v^2-v0^2)/(2*(x-x0))

In [ ]:
# # ve = 10u"km/s"
# # mi = 200u"kg"
# # mf = 80u"kg"
# # f = 2.4u"GHz"
# x0 = 1.0u"m"
# x = 2.0u"m"
# v0 = 1.0u"m/s"
# v = 2.0u"m/s"

# mods = Module[]
# AutoSolver.submodules(physics, mods)
# # mods = [Main]
# # mods = [AstroKit]

# outputs = AutoSolver.autoSolveOutputs(mods)


In [ ]:
# mods = Module[]
# funcs = Dict()
# AutoSolver.submodules(physics, mods)
# AutoSolver.moduleDict(Main.physics.kinematics, funcs)
# mods

In [ ]:
# out_d, in_d = AutoSolver.varExplodeDict(funcs)
# inp_map, out_map = AutoSolver.crossVarMap(in_d,out_d)
# inp_map

In [ ]:
# import Pkg; Pkg.add("XLSX")

In [ ]:
using XLSX

# Create Spreadsheet Template

In [ ]:
function autoSolveTemplate(mod::Module;filePath::String="",overwrite=false)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    mod_name = AutoSolver.modName(mod)
    
    if isfile("$(filePath)AS_$(mod_name).xlsx") && overwrite == false
        println("ERROR: $(filePath)AS_$(mod_name).xlsx already exists")
        return
    elseif !isdir(filePath) && filePath != ""
        mkdir(filePath)
    end

    XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="wr") do xf
        XLSX.addsheet!(xf)
        XLSX.addsheet!(xf)
        sh1 = xf[1]
        sh2 = xf[2]
        sh3 = xf[3]
        XLSX.rename!(sh1, "Input Map")
        XLSX.rename!(sh2, "Inputs")
        XLSX.rename!(sh3, "Outputs")

        sh1[1,1] = "Module"
        sh1[1,2] = "Function"
        sh1[1,3] = "Input Variable"
        sh1[1,4] = "Input Map"

        sh2[1,1] = "Input Variable"
        sh2[1,2] = "Input"
        sh2[1,3] = "Units"

        sh3[1,1] = "Module"
        sh3[1,2] = "Function"
        sh3[1,3] = "Output"
        sh3[1,4] = "Units"

        r1 = 2
        r2 = 2
        r3 = 2
        new_input = 0
        
        for submod in mods
            funcs = Dict()
            AutoSolver.moduleDict(submod, funcs)
            submod_name = AutoSolver.modName(submod)

            for func in funcs 
                sh1[r1,1] = submod_name
                sh1[r1,2] = string(func[1])

                for i = 1:length(func[2])
                    sh1[r1,3] = string(func[2][i])
                    sh1[r1,4] = string(func[2][i])
                    r1+=1

                    for j = 2:r2
                        if string(sh2[j,1]) == string(func[2][i])
                            new_input = 0
                            break
                        else
                            new_input = 1
                        end
                    end
                    if new_input == 1
                        sh2[r2,1] = string(func[2][i])
                        r2+=1
                    end
                end
            end

            for func in funcs      
                sh3[r3,1] = submod_name
                sh3[r3,2] = string(func[1])
                sh3[r3,3] = "-"
                sh3[r3,4] = "-"
                r3+=1
            end
        end
    end
end

In [ ]:
autoSolveTemplate(physics, filePath="C:/Users/ses/Documents/AutoSolver/", overwrite=false)

# Update Spreadsheet Mapping

In [ ]:
function inputMapFromXLSX(xf::XLSX.XLSXFile)
    sh = xf["Input Map"]
    inp_map = Dict()
    
    for (i,r) in enumerate(XLSX.eachrow(sh))
        if (i == 1 || ismissing(r[4])); continue; end
        if !haskey(inp_map, Symbol(r[3])); inp_map[Symbol(r[3])] = []; end
        if !(Symbol(r[4]) in inp_map[Symbol(r[3])]); push!(inp_map[Symbol(r[3])],Symbol(r[4])); end
    end

    return inp_map
end

In [ ]:
function inputsFromXLSX(xf::XLSX.XLSXFile)
    sh = xf["Inputs"]
    
    for (i,r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        if ismissing(@eval $(r[2])); continue; end
        if ismissing(@eval $(r[3]))
            @eval $(Symbol(r[1])) = (@eval $(r[2]))
        else
            @eval $(Symbol(r[1])) = (@eval $(r[2])) * (@eval @u_str $(r[3]))
        end
    end
end

In [ ]:
function outputToXLSX(xf::XLSX.XLSXFile, outputs::Dict)
    sh = xf["Outputs"]
  
    for (i, r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        func = Symbol(r[2])
        if haskey(outputs,func)
            sh[i,3] = string(ustrip(outputs[func]))
            sh[i,4] = string(unit(outputs[func]))
        else
            sh[i,3] = "-"
            sh[i,4] = "-"
        end
    end
end

In [ ]:
function autoSolveXLSX(mod::Module, filePath::String)
    inp_map = Dict()
    mod_name = AutoSolver.modName(mod)

    inp_map = XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="rw") do xf
        inp_map = inputMapFromXLSX(xf)
        inputsFromXLSX(xf)
        return inp_map
    end

    mods = Module[]
    AutoSolver.submodules(mod, mods)

    outputs = AutoSolver.autoSolveOutputs(mods, inp_map = inp_map)

    XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="rw") do xf
        outputToXLSX(xf,outputs)
    end
end

In [ ]:
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics

autoSolveXLSX(mod, filePath)

# AutoTester

In [ ]:
function autoTestTemplate(mod::Module;filePath::String="",overwrite=false)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    mod_name = AutoSolver.modName(mod)
    num_tests = 5
    
    if isfile("$(filePath)AT_$(mod_name).xlsx") && overwrite == false
        println("ERROR: $(filePath)AT_$(mod_name).xlsx already exists")
        return
    elseif !isdir(filePath) && filePath != ""
        mkdir(filePath)
    end
    
    if !isfile("$(filePath)AT_$(mod_name).xlsx")
        XLSX.openxlsx("$(filePath)AT_$(mod_name).xlsx", mode="w") do xf
        end
    end

    XLSX.openxlsx("$(filePath)AT_$(mod_name).xlsx", mode="rw") do xf
        sh1 = xf[1]
        XLSX.rename!(sh1, "Test Inputs")

        sh1[1,1] = "Module"
        sh1[1,2] = "Function"
        sh1[1,3] = "Input Vars"
        
        for i = 1:num_tests
            sh1[1,3*i+1] = "Inputs $(i)"
            sh1[1,3*i+2] = "Exp Output $(i)"
            sh1[1,3*i+3] = "Tol $(i) [%]"
        end

        r1 = 2
        new_input = 0
        
        for submod in mods
            funcs = Dict()
            AutoSolver.moduleDict(submod, funcs)
            submod_name = AutoSolver.modName(submod)

            for func in funcs
                args = string(func[2])
                sh1[r1,1] = submod_name
                sh1[r1,2] = string(func[1])
                sh1[r1,3] = string(strip(string(args[5:end]),[']']))
                r1+=1
            end
        end
    end
end

In [ ]:
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics

autoTestTemplate(mod, filePath=filePath, overwrite=false)

In [ ]:
function autoTestXLSX(mod::Module;filePath::String="")
    mod_name = AutoSolver.modName(mod)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    
    XLSX.openxlsx("$(filePath)AT_$(mod_name).xlsx", mode="rw") do xf
        sh = xf["Test Inputs"]
        
        for (i,r) in enumerate(XLSX.eachrow(sh))
            if i == 1; continue; end
            if ismissing(@eval $(r[4])); continue; end
#             if ismissing(@eval $(r[3]))
#                 @eval $(Symbol(r[1])) = (@eval $(r[2]))
#             else
#                 @eval $(Symbol(r[1])) = (@eval $(r[2])) * (@eval @u_str $(r[3]))
#             end
            println(r[1],", ",r[2],", [",r[4],"], ",r[5],", ",r[6])
            args = split(r[4],',')
            arg_arr = []
            for arg in args
                arg_in = split(strip(arg)," ")
                val = parse(Int,string(arg_in[1]))
                units = @eval @u_str $(arg_in[2])
                var = val * units
                push!(arg_arr,var)
            end
            
#             println(Module(Symbol(r[1])),".",Symbol(r[2]))
#             println("Inputs: ",arg_arr)
#             println("Expected Output: ",r[5])
            
            for mod_s in mods
                if string(mod_s) == string((@eval $(Module(Symbol(r[1])))))
                    output = AutoSolver.evalVariableInputs(mod_s,Symbol(r[2]),arg_arr)
                    println("Actual Output: ", output)
                    if string(output) == string(r[5])
                        println("Test Pass\n")
                    else
                        println("Test Failed\n")
                    end
                end
            end
#             result = (@eval $(Module(Symbol(r[1]))).$(Symbol(r[2]))($(arg_arr)...))
        end
    end
end

In [ ]:
using Test
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics

autoTestXLSX(mod, filePath=filePath)

In [ ]:
Main.physics.dynamics.F(1,2)
# Unitful.promote_to_derived()
# promote(1u"J",1.0u"kg*m^2/s^2")[2]

# AutoGenerate Database

In [ ]:
# using Pkg
# Pkg.add("PDFIO")

# """
# ​```
#     getPDFText(src, out) -> Dict 
# ​```
# - src - Input PDF file from where text is to be extracted
# - out - Output TXT file where the output will be written
# return - A dictionary containing metadata of the document
# """
# function getPDFText(src, out)
#     # handle that can be used for subsequence operations on the document.
#     doc = pdDocOpen(src)
    
#     # Metadata extracted from the PDF document. 
#     # This value is retained and returned as the return from the function. 
#     docinfo = pdDocGetInfo(doc) 
#     open(out, "w") do io
    
#         # Returns number of pages in the document       
#         npage = pdDocGetPageCount(doc)

#         for i=1:npage
        
#             # handle to the specific page given the number index. 
#             page = pdDocGetPage(doc, i)
            
#             # Extract text from the page and write it to the output file.
#             pdPageExtractText(io, page)

#         end
#     end
#     # Close the document handle. 
#     # The doc handle should not be used after this call
#     pdDocClose(doc)
#     return docinfo
# end

# using PDFIO
# getPDFText("electromagnetism_equations.pdf","test.txt")

In [103]:
using HTTP, Cascadia, Gumbo, AbstractTrees

url = "https://en.wikipedia.org/wiki/List_of_electromagnetism_equations"
r = HTTP.get(url) # Normally, you'd put a url here, but I couldn't find a way to grab it without having to download it and read it locally
h = parsehtml(String(r)) # Then normally you'd execute this

# # Instead, I'm going to read in the html file as a string and give it to Gumbo
# # h = parsehtml(read("em.html", String))

eqns = Dict()
index = 0
checkKey = false

# # Exploring with the various structure of Gumbo objects:
for elem in PreOrderDFS(h.root) 
    index = 0
    checkKey = false
    try
        eq = getattr(elem,"alt")
        eqn = split(eq,"=")
        println(eqn)
        println(fixEqn(eqn[2], eqnDict))
        if haskey(eqns,strip(fixEqn(eqn[1], eqnDict)))
            checkKey = true
            while checkKey == true
                index+=1
                eqnCheck = string(strip(fixEqn(eqn[1], eqnDict)),string(index))
                checkKey = haskey(eqns,eqnCheck)
            end
            eqn[1] = string(strip(fixEqn(eqn[1], eqnDict)),string(index))
        else
            eqn[1] = strip(fixEqn(eqn[1], eqnDict))
        end
        if occursin("\\",eqn[1])
            eqn[1] = string(latex2julia(eqn[1]))
        end
        eqn[2] = strip(fixEqn(eqn[2], eqnDict))
        eqns[eqn[1]] = latex2julia(eqn[2])
        println(eqns[eqn[1]])
    catch
    end
end

@show eqns

# println(fieldnames(h.root))
# println(fieldnames(h.root.children))
# println(size(h.root.children))

# aviks code:
# c = matchall(Selector("td:containsOwn(\"CAN:\") + td span"), h.root);
# for x in c
#     println( x.children[1].text )
# end

SubString{String}["Solenoid"]
SubString{String}[""]
SubString{String}[""]
SubString{String}[" q_e ", " \\int \\lambda_e \\mathrm{d}\\ell "]
 \int \lambda_e d\ell 
Integral(lambda_{e}, (:ell,))
SubString{String}[" q_e ", " \\iint \\sigma_e \\mathrm{d} S "]
 \iint \sigma_e d S 
iint * (sigma_{e} * dS)
SubString{String}[" q_e ", " \\iiint \\rho_e \\mathrm{d}V "]
 \iiint \rho_e dV 
iiint * (rho_{e} * dV)
SubString{String}["C ", " \\mathrm{d}q/\\mathrm{d}V\\,\\!"]
 dq/dV
dq * dV ^ -1
SubString{String}[" I ", " \\mathrm{d}q/\\mathrm{d}t \\,\\!"]
 dq/dt 
dq * dt ^ -1
SubString{String}["I ", " \\mathbf{J} \\cdot \\mathrm{d} \\mathbf{S}"]
 J \cdot d S
J * dS
SubString{String}[" \\mathbf{J}_\\mathrm{d} ", " \\epsilon_0 \\left ( \\partial \\mathbf{E} / \\partial t \\right ) ", " \\partial \\mathbf{D} / \\partial t \\,\\!"]
 \epsilon_0 ( ( d E / d t ) ) 
epsilon_{0}(dE * dt ^ -1)
SubString{String}[" \\mathbf{J}_\\mathrm{c} ", " \\rho \\mathbf{v} \\,\\!"]
 \rho v 
rho * v
SubString{String}["\\mathb

mu_{0} * (4pi) ^ -1
SubString{String}["\\mathbf{B}({\\mathbf{r}})", "\\nabla\\times{\\mathbf{A}}", "\\frac{\\mu_{0}}{4\\pi}\\left(\\frac{3\\mathbf{r}(\\mathbf{m}\\cdot\\mathbf{r})}{\\left | \\mathbf{r} \\right |^{5}}-\\frac{{\\mathbf{m}}}{\\left | \\mathbf{r} \\right |^{3}}\\right)"]
\nabla\times{A}
nabla * A
SubString{String}[" \\mathbf{m} ", " \\frac{1}{2}\\int_V \\mathbf{r}\\times\\mathbf{J} \\mathrm{d} V "]
 \frac{1}{2}\int_V r\timesJ d V 
2 ^ -1
SubString{String}[" \\boldsymbol{\\tau} ", " \\int_V \\mathrm{d} \\mathbf{m} \\times \\mathbf{B} "]
 \int_V d m \times B 
SubString{String}[" U ", " \\int_V \\mathrm{d} \\mathbf{m} \\cdot \\mathbf{B} "]
 \int_V d m \cdot B 
SubString{String}["\\frac{V_s}{V_p} ", " \\frac{N_s}{N_p} ", " \\frac{I_p}{I_s} ", " \\eta \\,\\!"]
 \frac{N_s}{N_p} 
N_{s} * N_{p} ^ -1
SubString{String}["R_\\mathrm{net} ", " \\sum_{i", "1}^{N} R_i\\,\\!"]
 \sum_{i
SubString{String}["{1 \\over G_{{\\mathrm  {net}}}}", "\\sum _{{i", "1}}^{{N}}{1 \\over G_{i}}\\,\\!"]
\

In [ ]:
# ENV["PYTHON"] = "C:/Users/ses/.julia/conda/3/python"
# using Pkg
# Pkg.build("PyCall")
using PyCall

In [5]:
using SymPy

In [6]:
# platex = SymPy.PyCall.pyimport("sympy.parsing.latex")

In [7]:
const sympy_parsing_mathematica = SymPy.PyCall.pyimport("sympy.parsing.mathematica")
mathematica2julia(s::AbstractString, substitutions::Pair{<:AbstractString,<:AbstractString}...) =
           SymPy.walk_expression(sympy_parsing_mathematica."mathematica"(s, Dict(substitutions...)))

mathematica2julia (generic function with 1 method)

In [8]:
mathematica2julia("Log[Log[x]]")

:(log(log(x)))

In [9]:
const sympy_parse_latex = SymPy.PyCall.pyimport("sympy.parsing.latex")
latex2julia(s::AbstractString) = SymPy.walk_expression(sympy_parse_latex."parse_latex"(s))

latex2julia (generic function with 1 method)

In [10]:
function Integral(f,d;min=0.0,max="")
    if max == ""
        integrate(f,symbols(string(d...)))
    else
        integrate(f,min,max)
    end
end

l = symbols("l")
lambda1(l) = l^2

# @eval $(latex2julia("\\int \\lambda dl"))

lambda1 (generic function with 1 method)

In [11]:
Integral(lambda1,(:l,),max=10.0)

333.333333333333

In [63]:
function fixEqn(eqn,d)
    for key in d
        eqn = replace(eqn, key[1] => key[2])
    end
    return eqn
end

fixEqn (generic function with 1 method)

In [102]:
eqnDict = Dict(
    r"\\mathrm{(.)}" => s"\1",
    r"\\mathbf{(.)}" => s"\1",
    r"\\mathcal{(.)}" => s"\1",
    "\\left" => "(",
    "\\right" => ")",
    r"\\partial" => s"d",
    "\\,\\!" => "",
    "\\," => "",
    "\\langle" => "(",
    "\\rangle" => ")",
    "int_S" => "int",
)

Dict{Any,AbstractString} with 11 entries:
  "int_S"           => "int"
  "\\rangle"        => ")"
  "\\,\\!"          => ""
  "\\left"          => "("
  r"\\partial"      => s"d"
  r"\\mathcal{(.)}" => s"\\1"
  "\\right"         => ")"
  r"\\mathbf{(.)}"  => s"\\1"
  "\\,"             => ""
  "\\langle"        => "("
  r"\\mathrm{(.)}"  => s"\\1"

In [75]:
eqn = "\\int \\lambda_e \\mathbf{T} \\mathrm{d}\\ell\\,\\!"
println(eqn)
eqn = fixEqn(eqn,eqnDict)
latex2julia(eqn)

\int \lambda_e \mathbf{T} \mathrm{d}\ell\,\!


:(Integral(lambda_{e} * T, (:ell,)))